# Cartographie Interactive des Lignes de Métro en Île-de-France


Le jeu de données utilisé pour cette analyse provient du site [data.iledefrance-mobilites.fr](https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/traces-du-reseau-ferre-idf/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B), qui fournit les tracés du réseau ferré en Île-de-France. Ce dataset contient des informations géolocalisées sur les lignes de train, de RER, de métro et de tramway, y compris les exploitants, les dates de mise en service et d'autres détails techniques.

L'objectif de cette analyse est de préparer les données pour créer une carte interactive des lignes de métro en Île-de-France, en incluant les couleurs web associées à chaque ligne. Cela permettra une visualisation claire et attrayante des lignes de métro dans la région.


## Méthodologie

La méthodologie utilisée pour cette analyse implique plusieurs étapes clés. Tout d'abord, nous avons sélectionné les données pertinentes en utilisant une requête SQL pour extraire les informations nécessaires sur les lignes de métro. Ensuite, nous avons converti les géométries des lignes en objets Shapely pour faciliter leur utilisation dans la création de la carte interactive. Finalement, nous avons utilisé la bibliothèque Folium pour créer la carte, en ajoutant les lignes de métro avec leurs couleurs respectives et en incluant une légende et un titre pour améliorer la lisibilité de la visualisation.


Nous avons utilisé une requête SQL pour sélectionner les données pertinentes, en nous concentrant sur les lignes de métro et en extrayant les informations sur leur géométrie et leurs couleurs web. Les données ont ensuite été traitées et visualisées à l'aide de bibliothèques Python telles que DuckDB, Pandas et Folium.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/traces-du-reseau-ferre-idf/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
  ST_AsText(ST_GeomFromGeoJSON("Geo Shape")) AS geom_wkt,
  "mode",
  "ColourWeb_hexa" AS couleur_web
FROM 
  loaded_dataset
WHERE 
  "mode" = 'METRO' """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Folium, qui est une surcouche Python pour Leaflet.js, permettant de créer des cartes interactives. Folium est adaptée pour visualiser des données géospatiales, comme les lignes de métro, sur une carte interactive. Cela permet une représentation visuelle claire et dynamique des données.

In [3]:
import pandas as pd
import duckdb as ddb
import folium
from shapely import wkt
import pandas as pd

# Assurez-vous que df contient déjà les colonnes geom_wkt, couleur_web et mode
# Conversion des géométries WKT en objets Shapely
gdf = df.copy()
gdf['geometry'] = gdf['geom_wkt'].apply(wkt.loads)

# Création de la carte centrée sur Paris
m = folium.Map(
    location=[48.8566, 2.3522],
    zoom_start=12,
    tiles='cartodbpositron',
    attr='© OpenStreetMap contributors © CARTO'
)

group = folium.FeatureGroup(name='Lignes de métro', overlay=True, control=True)

# Ajout des lignes colorées
for _, row in gdf.iterrows():
    color = '#' + row['couleur_web'].lstrip('#')
    line = folium.PolyLine(
        locations=[[lat, lon] for lon, lat in wkt.loads(row['geom_wkt']).coords],
        color=color,
        weight=5,
        opacity=0.8,
        tooltip=f"Mode : {row['mode']}<br>Couleur : {color}"
    )
    line.add_to(group)
group.add_to(m)

# Légende HTML
legend_code = '''
<div style="
    position: fixed; 
    bottom: 20px; left: 20px; width: 220px; height: 50px; 
    background-color: white; z-index: 900; border:2px solid grey;
    padding: 8px;
    font-family: 'Roboto', sans-serif;
    font-size: 14px;
">
<b>Légende :</b><br>
Lignes de métro<br>
(Modes et couleurs issues des données)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_code))

# Titre HTML
title_code = '''
<div style="
    position: fixed; 
    top: 10px; left: 50%; transform: translateX(-50%);
    background-color: rgba(255,255,255,0.9); z-index: 900; border:2px solid grey;
    padding: 8px 16px;
    font-family: 'Roboto', sans-serif;
    font-size: 18px;
    font-weight: bold;
">
Carte interactive des lignes de métro d'Île-de-France
</div>
'''
m.get_root().html.add_child(folium.Element(title_code))

folium.LayerControl(collapsed=False).add_to(m)

# Variable globale pour publication
dataviz = m
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*